In [5]:
import os
import smtplib
from datetime import datetime
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

def send_email_with_attachment(recipients, subject, body, attachment_path):
    fromaddr = "reports@scalptrade.com"
    toaddr = recipients

    msg = MIMEMultipart()
    msg['From'] = fromaddr
    msg['To'] = ", ".join(toaddr)
    msg['Subject'] = subject

    # Attach the body with the msg instance
    msg.attach(MIMEText(body, 'plain'))

    # Open the file to be sent
    filename = os.path.basename(attachment_path)
    attachment = open(attachment_path, "rb")

    # Instance of MIMEBase and named as p
    p = MIMEBase('application', 'octet-stream')

    # To change the payload into encoded form
    p.set_payload(attachment.read())

    # Encode into base64
    encoders.encode_base64(p)

    p.add_header('Content-Disposition', f"attachment; filename= {filename}")

    # Attach the instance 'p' to instance 'msg'
    msg.attach(p)

    # Use the SMTP server and send the email
    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.starttls()
    s.login(fromaddr, "sc@lptrade")
    text = msg.as_string()
    s.sendmail(fromaddr, toaddr, text)
    s.quit()

def scp_file_to_local(remote_path, local_path, remote_user, remote_host, password):
    command = f"sshpass -p '{password}' scp {remote_user}@{remote_host}:{remote_path} {local_path}"
    os.system(command)

def main():
    # Construct the filename with the current date
    today = datetime.now().strftime("%Y%m%d")
    filename = f"MPTakingStats_{today}.csv"
    remote_path = os.path.join("/home/scalp/MPTaking", filename)
    local_path = os.path.join("./", filename)
    
    password = "QAtr@de442"
    scp_file_to_local(remote_path, local_path, 'scalp', '10.7.8.114', password)


    # Check if the file exists
    if os.path.exists(local_path):
        team_alpha = ["ewashington@scalptrade.com"]
        send_email_with_attachment(team_alpha, "MPTaking Stats for Today", "Please find attached the MPTaking stats for today.", local_path)
    else:
        print(f"File {filename} does not exist!")

#if __name__ == "__main__":
main()